In [7]:
## Install Dependencies

!pip install streamlit pyngrok scikit-learn pandas numpy


In [8]:
### Download or Upload Dataset

!wget -q https://raw.githubusercontent.com/akshitagupta15june/ML-Crop-Recommendation-Dataset/master/Crop_recommendation.csv


In [9]:
#### Train the Crop Recommendation Model

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle

# Load dataset
data = pd.read_csv("Crop_recommendation.csv")

X = data.drop('label', axis=1)
y = data['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

print("✅ Model trained successfully!")

# Save the model
with open("crop_model.pkl", "wb") as f:
    pickle.dump(model, f)


✅ Model trained successfully!


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
from pyngrok import ngrok

# 🔑 Replace YOUR_AUTHTOKEN with your actual token from ngrok dashboard
ngrok.set_auth_token("358nv8kHwf6bJKy6FxmzSPVRfV8_7A7VZpu7N2y7RCwbkn6gz")


In [14]:
#### Create the Streamlit App File

%%writefile app.py
import streamlit as st
import numpy as np
import pickle

st.title("🌾 AI Crop & Fertilizer Recommendation System")

N = st.number_input("Nitrogen (N)", 0, 150)
P = st.number_input("Phosphorus (P)", 0, 150)
K = st.number_input("Potassium (K)", 0, 150)
temperature = st.number_input("Temperature (°C)", 0.0, 50.0)
humidity = st.number_input("Humidity (%)", 0.0, 100.0)
ph = st.number_input("pH", 0.0, 14.0)
rainfall = st.number_input("Rainfall (mm)", 0.0, 500.0)

@st.cache_resource
def load_model():
    with open("crop_model.pkl", "rb") as f:
        return pickle.load(f)

model = load_model()

def recommend_fertilizer(N, P, K):
    if N < 50:
        return "Add Nitrogen-rich fertilizer (Urea)"
    elif P < 50:
        return "Add Phosphorus-rich fertilizer (DAP)"
    elif K < 50:
        return "Add Potassium-rich fertilizer (MOP)"
    else:
        return "Balanced fertilizer (NPK 10:10:10)"

if st.button("Recommend Crop & Fertilizer"):
    crop = model.predict([[N, P, K, temperature, humidity, ph, rainfall]])[0]
    fert = recommend_fertilizer(N, P, K)
    st.success(f"🌱 Recommended Crop: **{crop}**")
    st.info(f"💧 Fertilizer Suggestion: **{fert}**")


Writing app.py


In [15]:
#### Run the Streamlit App in Colab

from pyngrok import ngrok
import threading, subprocess

# Replace 'YOUR_AUTHTOKEN' with your actual ngrok authtoken
# You can get one from https://dashboard.ngrok.com/get-started/your-authtoken
ngrok.set_auth_token("358nv8kHwf6bJKy6FxmzSPVRfV8_7A7VZpu7N2y7RCwbkn6gz")

def run_streamlit():
    subprocess.run(["streamlit", "run", "app.py", "--server.port", "8501"])

thread = threading.Thread(target=run_streamlit)
thread.start()

# Create a public URL
public_url = ngrok.connect(8501)
print("Your app is live at:", public_url)

Your app is live at: NgrokTunnel: "https://audria-postarytenoid-uncorruptly.ngrok-free.dev" -> "http://localhost:8501"
